# Trabalho Prático 1 - Compressão de imagens
## Introdução à Computação Visual - 2020/1
#### Integrantes:
* Otávio Augusto Silva - 2016006808
* Luiz Henrique de Melo Santos - 2017014464

Neste Trabalho Prático será feita a implementação de uma aplicação para compressão de imagens sem perdas, por meio do Algoritmo de Huffman.

In [1]:
# para utilizar a aplicacao execute
# $ conda install -c menpo imageio
# $ conda install -c conda-forge pillow

import cv2
import sys
import copy
import time
import queue
import string
import numpy as np
from itertools import chain
from collections import Counter
from PIL import Image,ImageChops
from matplotlib import pyplot as plt
from operator import itemgetter, attrgetter

Definição da estrutura de dados e funções auxiliares que serão utilizados para a criação e para o uso da árvore exigida pelo Algoritmo.

In [2]:
# classe que representara uma unidade de no na arvore
class Node:
    def __init__(self):
        self.prob = None
        self.code = None
        self.data = None
        self.left = None
        self.right = None
    def __lt__(self, other):
        if (self.prob < other.prob):  # promove uma classificacao na fila de prioridades
            return 1
        else:
            return 0
    def __ge__(self, other):
        if (self.prob > other.prob):
            return 1
        else:
            return 0

In [3]:
def rgb2gray(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray_img

In [4]:
def get2smallest(data):  # can be used instead of inbuilt function get(). was not used in implementation
    first = second = 1
    fid = sid = 0
    for idx, element in enumerate(data):
        if (element < first):
            second = first
            sid = fid
            first = element
            fid = idx
        elif (element < second and element != first):
            second = element
    return fid, first, sid, second

In [5]:
def tree(probabilities):
    prq = queue.PriorityQueue()
    for color, probability in enumerate(probabilities):
        leaf = Node()
        leaf.data = color
        leaf.prob = probability
        prq.put(leaf)
    while (prq.qsize() > 1):
        newnode = Node()  # criacao de um novo no
        l = prq.get()
        r = prq.get()  # identifica as menores probabilidades na arvore
        # remove os dois menores nos
        newnode.left = l  # no da esquerda sera o menor
        newnode.right = r
        newprob = l.prob + r.prob  #  a nova probabilidade no novo no deve ser a soma dos outros dois
        newnode.prob = newprob
        prq.put(newnode)  #  o novo no eh inserido, substituindo os outros dois
    return prq.get()  #  retorna o caminho de nos - a arvore esta completa

In [6]:
def huffman_traversal(root_node,tmp_array,f):  # traversal of the tree to generate codes
    if (root_node.left is not None):
        tmp_array[huffman_traversal.count] = 1
        huffman_traversal.count += 1
        huffman_traversal(root_node.left,tmp_array,f)
        huffman_traversal.count -= 1
    if (root_node.right is not None):
        tmp_array[huffman_traversal.count] = 0
        huffman_traversal.count += 1
        huffman_traversal(root_node.right,tmp_array,f)
        huffman_traversal.count -= 1
    else: 
        huffman_traversal.output_bits[root_node.data] = huffman_traversal.count  # promove a contagem de bits para uma determinada cor
        bitstream = ''.join(str(cell) for cell in tmp_array[1:huffman_traversal.count]) 
        color = str(root_node.data)
        wr_str = color + ' ' + bitstream + '\n'
        f.write(wr_str)  # escreve a cor e o codigo em um arquivo externo

In [7]:
# Read an bmp image into a numpy array
img = cv2.imread('tiger.bmp')

In [8]:
# convert to grayscale
gray_img = rgb2gray(img)

In [9]:
# compute histogram of pixels
hist = np.bincount(gray_img.ravel(),minlength=256)

In [10]:
probabilities = hist / np.sum(hist)  # a priori probabilities from frequencies

In [11]:
root_node = tree(probabilities)  # create the tree using the probs.
tmp_array = np.ones([64],dtype=int)
huffman_traversal.output_bits = np.empty(256,dtype=int) 
huffman_traversal.count = 0
f = open('codes.txt','w')
huffman_traversal(root_node, tmp_array, f)  # traverse the tree and write the codes

In [12]:
input_bits = np.shape(img)[0]* np.shape(img)[1] * 8  # calculate number of bits in grayscale 
compression = (1 - np.sum(huffman_traversal.output_bits *hist) / input_bits) * 100  # compression rate
print('Compression is ',compression,' percent')

Compression is  12.713601919110395  percent


In [13]:
def count_symbols(image):
    pixels = image.getdata()
    values = chain.from_iterable(pixels)
    counts = Counter(values).items()
    return sorted(counts, key=lambda x:x[::-1])

In [14]:
def build_tree(counts) :
    nodes = [entry[::-1] for entry in counts] # Reverse each (symbol,count) tuple
    while len(nodes) > 1 :
        leastTwo = tuple(nodes[0:2]) # get the 2 to combine
        theRest = nodes[2:] # all the others
        combFreq = leastTwo[0][0] + leastTwo[1][0]  # the branch points freq
        nodes = theRest + [(combFreq, leastTwo)] # add branch point to the end
        nodes.sort(key=lambda t: t[0]) # sort it into place
    return nodes[0] # Return the single tree inside the list

In [15]:
def trim_tree(tree) :
    p = tree[1] # Ignore freq count in [0]
    if type(p) is tuple: # Node, trim left then right and recombine
        return (trim_tree(p[0]), trim_tree(p[1]))
    return p # Leaf, just return it

In [16]:
def assign_codes_impl(codes, node, pat):
    if type(node) == tuple:
        assign_codes_impl(codes, node[0], pat + [0]) # Branch point. Do the left branch
        assign_codes_impl(codes, node[1], pat + [1]) # then do the right branch.
    else:
        codes[node] = pat # A leaf. set its code

def assign_codes(tree):
    codes = {}
    assign_codes_impl(codes, tree, [])
    return codes

In [17]:
def to_binary_list(n):
    """Convert integer into a list of bits"""
    return [n] if (n <= 1) else to_binary_list(n >> 1) + [n & 1]

def from_binary_list(bits):
    """Convert list of bits into an integer"""
    result = 0
    for bit in bits:
        result = (result << 1) | bit
    return result

def pad_bits(bits, n):
    """Prefix list of bits with enough zeros to reach n digits"""
    assert(n >= len(bits))
    return ([0] * (n - len(bits)) + bits)

In [18]:
class OutputBitStream: 
    def __init__(self, file_name): 
        self.file_name = file_name
        self.file = open(self.file_name, 'wb') 
        self.bytes_written = 0
        self.buffer = []

    def write_bit(self, value):
        self.write_bits([value])

    def write_bits(self, values):
        self.buffer += values
        while len(self.buffer) >= 8:
            self._save_byte()        

    def flush(self):
        if len(self.buffer) > 0: # Add trailing zeros to complete a byte and write it
            self.buffer += [0] * (8 - len(self.buffer))
            self._save_byte()
        assert(len(self.buffer) == 0)

    def _save_byte(self):
        bits = self.buffer[:8]
        self.buffer[:] = self.buffer[8:]

        byte_value = from_binary_list(bits)
        self.file.write(bytes([byte_value]))
        self.bytes_written += 1

    def close(self): 
        self.flush()
        self.file.close()

In [19]:
class InputBitStream: 
    def __init__(self, file_name): 
        self.file_name = file_name
        self.file = open(self.file_name, 'rb') 
        self.bytes_read = 0
        self.buffer = []

    def read_bit(self):
        return self.read_bits(1)[0]

    def read_bits(self, count):
        while len(self.buffer) < count:
            self._load_byte()
        result = self.buffer[:count]
        self.buffer[:] = self.buffer[count:]
        return result

    def flush(self):
        assert(not any(self.buffer))
        self.buffer[:] = []

    def _load_byte(self):
        value = ord(self.file.read(1))
        self.buffer += pad_bits(to_binary_list(value), 8)
        self.bytes_read += 1

    def close(self): 
        self.file.close()

In [21]:
def compressed_size(counts, codes):
    header_size = 2 * 16 # height and width as 16 bit values

    tree_size = len(counts) * (1 + 8) # Leafs: 1 bit flag, 8 bit symbol each
    tree_size += len(counts) - 1 # Nodes: 1 bit flag each
    if tree_size % 8 > 0: # Padding to next full byte
        tree_size += 8 - (tree_size % 8)

    # Sum for each symbol of count * code length
    pixels_size = sum([count * len(codes[symbol]) for symbol, count in counts])
    if pixels_size % 8 > 0: # Padding to next full byte
        pixels_size += 8 - (pixels_size % 8)

    return (header_size + tree_size + pixels_size) / 8

In [ ]:
def encode_header(image, bitstream):
    height_bits = pad_bits(to_binary_list(image.height), 16)
    bitstream.write_bits(height_bits)    
    width_bits = pad_bits(to_binary_list(image.width), 16)
    bitstream.write_bits(width_bits)

In [ ]:
def encode_tree(tree, bitstream):
    if type(tree) == tuple: # Note - write 0 and encode children
        bitstream.write_bit(0)
        encode_tree(tree[0], bitstream)
        encode_tree(tree[1], bitstream)
    else: # Leaf - write 1, followed by 8 bit symbol
        bitstream.write_bit(1)
        symbol_bits = pad_bits(to_binary_list(tree), 8)
        bitstream.write_bits(symbol_bits)

In [ ]:
def encode_pixels(image, codes, bitstream):
    for pixel in image.getdata():
        for value in pixel:
            bitstream.write_bits(codes[value])

In [ ]:
def compress_image(in_file_name, out_file_name):
    print('Compressing "%s" -> "%s"' % (in_file_name, out_file_name))
    image = Image.open(in_file_name)
    print('Image shape: (height=%d, width=%d)' % (image.height, image.width))
    size_raw = raw_size(image.height, image.width)
    print('RAW image size: %d bytes' % size_raw)
    counts = count_symbols(image)
    print('Counts: %s' % counts)
    tree = build_tree(counts)
    print('Tree: %s' % str(tree))
    trimmed_tree = trim_tree(tree)
    print('Trimmed tree: %s' % str(trimmed_tree))
    codes = assign_codes(trimmed_tree)
    print('Codes: %s' % codes)
    size_estimate = compressed_size(counts, codes)
    print('Estimated size: %d bytes' % size_estimate)
    print('Writing...')
    stream = OutputBitStream(out_file_name)
    print('* Header offset: %d' % stream.bytes_written)
    encode_header(image, stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Tree offset: %d' % stream.bytes_written)
    encode_tree(trimmed_tree, stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Pixel offset: %d' % stream.bytes_written)
    encode_pixels(image, codes, stream)
    stream.close()
    size_real = stream.bytes_written
    print('Wrote %d bytes.' % size_real)
    print('Estimate is %scorrect.' % ('' if size_estimate == size_real else 'in'))
    print('Compression ratio: %0.2f' % (float(size_raw) / size_real))

In [22]:
def decode_header(bitstream):
    height = from_binary_list(bitstream.read_bits(16))
    width = from_binary_list(bitstream.read_bits(16))
    return (height, width)

# https://stackoverflow.com/a/759766/3962537
def decode_tree(bitstream):
    flag = bitstream.read_bits(1)[0]
    if flag == 1: # Leaf, read and return symbol
        return from_binary_list(bitstream.read_bits(8))
    left = decode_tree(bitstream)
    right = decode_tree(bitstream)
    return (left, right)

def decode_value(tree, bitstream):
    bit = bitstream.read_bits(1)[0]
    node = tree[bit]
    if type(node) == tuple:
        return decode_value(node, bitstream)
    return node

def decode_pixels(height, width, tree, bitstream):
    pixels = bytearray()
    for i in range(height * width * 3):
        pixels.append(decode_value(tree, bitstream))
    return Image.frombytes('RGB', (width, height), bytes(pixels))

def decompress_image(in_file_name, out_file_name):
    print('Decompressing "%s" -> "%s"' % (in_file_name, out_file_name))

    print('Reading...')
    stream = InputBitStream(in_file_name)
    print('* Header offset: %d' % stream.bytes_read)
    height, width = decode_header(stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Tree offset: %d' % stream.bytes_read)    
    trimmed_tree = decode_tree(stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Pixel offset: %d' % stream.bytes_read)
    image = decode_pixels(height, width, trimmed_tree, stream)
    stream.close()
    print('Read %d bytes.' % stream.bytes_read)

    print('Image size: (height=%d, width=%d)' % (height, width))
    print('Trimmed tree: %s' % str(trimmed_tree))
    image.save(out_file_name)

In [23]:
def raw_size(width, height):
    header_size = 2 * 16 # height and width as 16 bit values
    pixels_size = 3 * 8 * width * height # 3 channels, 8 bits per channel
    return (header_size + pixels_size) / 8

def images_equal(file_name_a, file_name_b):
    image_a = Image.open(file_name_a)
    image_b = Image.open(file_name_b)

    diff = ImageChops.difference(image_a, image_b)

    return diff.getbbox() is None

if __name__ == '__main__':
    start = time.time()

    compress_image('tiger.png', 'answer.txt')

    print('-' * 40)

    decompress_image('answer.txt', 'out.png')

    stop = time.time()
    times = (stop - start) * 1000

    print('-' * 40)

    print('Run time takes %d miliseconds' % times)
    print('Images equal = %s' % images_equal('tiger.png', 'out.png'))

Compressing "tiger.png" -> "answer.txt"
Image shape: (height=354, width=630)
RAW image size: 669064 bytes
Counts: [(246, 243), (242, 254), (243, 256), (240, 257), (241, 272), (245, 285), (244, 290), (237, 294), (235, 295), (238, 300), (239, 301), (236, 305), (247, 306), (233, 307), (234, 313), (248, 316), (230, 329), (232, 340), (225, 351), (249, 355), (231, 360), (227, 363), (229, 363), (226, 364), (255, 367), (228, 373), (250, 381), (223, 391), (215, 405), (219, 406), (224, 406), (222, 407), (218, 415), (220, 427), (221, 429), (214, 434), (251, 436), (213, 452), (217, 460), (208, 464), (216, 467), (253, 468), (206, 481), (211, 481), (210, 484), (254, 490), (209, 491), (212, 494), (252, 501), (207, 503), (204, 512), (205, 518), (200, 530), (198, 534), (201, 554), (194, 559), (202, 561), (196, 563), (203, 572), (197, 574), (191, 580), (192, 584), (195, 587), (193, 596), (190, 597), (185, 609), (199, 622), (189, 628), (188, 631), (187, 633), (184, 650), (186, 655), (181, 684), (183, 697